In [4]:
# --- Notebook: image_client.ipynb ---
# Cell 1: Install deps
%pip -q install requests pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Cell 2: Imports + helpers
from __future__ import annotations

import base64
import io
from pathlib import Path

import requests
from PIL import Image


def request_image(
    server_url: str,
    size: int,
    contents: str,
    timeout_s: float = 10.0,
) -> dict:
    """
    Returns parsed JSON response.
    """
    payload = {"size": size, "contents": contents}
    r = requests.post(f"{server_url.rstrip('/')}/render", json=payload, timeout=timeout_s)
    r.raise_for_status()
    return r.json()


def save_base64_png(b64_png: str, out_path: str | Path) -> Path:
    out_path = Path(out_path)
    png_bytes = base64.b64decode(b64_png)

    # sanity-check load via Pillow
    img = Image.open(io.BytesIO(png_bytes))
    img.load()

    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("wb") as f:
        f.write(png_bytes)

    return out_path


In [6]:
# Cell 3: Call server + save to file
from datetime import datetime

SERVER = "http://127.0.0.1:8000"
SIZE   = 128
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
SPEC   = f"text:Hello from client \n@ {timestamp}"

try:
    resp = request_image(SERVER, SIZE, SPEC)
    print({k: resp[k] for k in ("width", "height", "mime")})

    out = save_base64_png(resp["rgba_png_base64"], f"out_{SIZE}.png")
    print(f"Saved: {out.resolve()}")
except requests.exceptions.ConnectionError:
    print(f"ERROR: Cannot connect to server at {SERVER}")
    print("Make sure the server notebook is running first!")
except requests.exceptions.HTTPError as e:
    print(f"ERROR: Server returned an error: {e}")
    print("Check the server notebook for errors.")
except Exception as e:
    print(f"ERROR: {type(e).__name__}: {e}")


{'width': 128, 'height': 128, 'mime': 'image/png'}
Saved: C:\Util\Data\GitRepos\KoreCommon\out_128.png
